<a href="https://colab.research.google.com/github/linxizhu1209/AI-Repo/blob/main/PDF%E1%84%85%E1%85%B3%E1%86%AF_%E1%84%92%E1%85%A1%E1%86%A8%E1%84%89%E1%85%B3%E1%86%B8%E1%84%92%E1%85%A1%E1%86%AB_%E1%84%82%E1%85%A1%E1%84%86%E1%85%A1%E1%86%AB%E1%84%8B%E1%85%B4_%E1%84%8E%E1%85%A2%E1%86%BA%E1%84%87%E1%85%A9%E1%86%BA_%E1%84%86%E1%85%A1%E1%86%AB%E1%84%83%E1%85%B3%E1%86%AF%E1%84%80%E1%85%B5_(FAISS_%E1%84%92%E1%85%AA%E1%86%AF%E1%84%8B%E1%85%AD%E1%86%BC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG 실습 : PDF를 학습한 나만의 챗봇 만들기 (FAISS 활용)
  - 데이터 로드(Load Data)
    - RAG에 사용할 데이터를 불러오는 단계
  - 텍스트 분할(Text Split)
    - 불러온 데이터를 작은 크기의 단위(chunk)로 분할하는 단계
  - 임베딩 (Embedding) / 인덱싱 (Indexing)
    - 텍스트 데이터를 숫자로 이루어진 벡터로 변환하는 단계
    - 분할된 텍스트를 검색 가능한 형태로 만드는 단계
  - 검색(Retrieval)
    - 사용자의 질문이나 주어진 컨텍스트에 가장 관련된 정보를 찾아내는 단계
  - 생성(Generation)
    - 검색된 정보를 바탕으로 사용자의 질문에 답변을 생성하는 최종 단계



In [1]:
!pip install openai faiss-cpu pypdf
!pip install langchain==0.3.27
!pip install langchain_community==0.3.31
!pip install langchain_core==0.3.79
!pip install langchain_experimental==0.3.4
!pip install langchain_huggingface==0.3.0
!pip install langchain_openai==0.3.28
!pip install langchain_text_splitters==0.3.11


!pip list | grep langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.8/449.8 kB 24.9 MB/s eta 0:00:00
  Attempting uninstall: langchain_core
    Found existing installation: langchain-core 0.3.83
    Uninstalling langchain-core-0.3.83:
      Successfully uninstalled langchain-core-0.3.83
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langgraph-prebuilt 1.0.7 requires langchain-core>=1.0.0, but you have langchain-core 0.3.79 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.6/948.6 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 2.21.0
    Uninstalling openai-2.21.0:
      Successfully uninstalled openai-2.21.0
langchain                                0.3.27

In [14]:
# API_KEY 등록

import os
# from google.colab import userdata

# 환경변수 등록
# os.environ['OPENAI_API_KEY'] = userdata.get("OPENAI_API_KEY")

In [15]:
# 모델 생성 및 질의하기
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0, max_tokens = 1024)

In [10]:
print( llm.invoke("일본의 경제 성장률이 어때").content )

2023년 기준으로 일본의 경제 성장률은 여러 요인에 따라 변동성이 있습니다. 일본은 고령화 사회와 인구 감소, 그리고 지속적인 디플레이션 문제 등으로 인해 경제 성장에 어려움을 겪고 있습니다. 그러나 정부의 경제 정책, 글로벌 경제 상황, 그리고 COVID-19 팬데믹 이후의 회복 상황 등이 일본 경제에 영향을 미치고 있습니다.

2023년에는 일본 경제가 회복세를 보일 것으로 예상되었지만, 여전히 낮은 성장률을 기록할 가능성이 높습니다. 일본 정부는 경제 성장을 촉진하기 위해 다양한 정책을 시행하고 있으며, 특히 기술 혁신과 디지털 전환에 중점을 두고 있습니다.

정확한 성장률 수치는 경제 지표와 보고서에 따라 다를 수 있으므로, 최신 경제 데이터를 확인하는 것이 중요합니다.


### 데이터 로드(Load Data)
- RAG 에 사용할 데이터를 불러오는 단계
  - PDF 파일 불러오기



In [20]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

#############################################
## 문제 1: 데이터 로드 및 청크 생성
## RecursiveCharacterTextSplitter, PyPDFLoader를 사용해서 문서 파싱 및 청크 생성

FILE_PATH = "현안과과제_주요국 경제 및 주요 가격지표 전망과 시사점_240115.pdf"
loader = PyPDFLoader(FILE_PATH)

docs = loader.load()
len(docs)

splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=0)

chunks = loader.load_and_split(splitter)

print(f"문서의 길이: {len(chunks)}")


문서의 길이: 154


Document(metadata={'producer': 'Hancom PDF 1.3.0.534', 'creator': 'Hwp 2018 11.0.0.1623', 'creationdate': '2024-05-22T17:27:21+09:00', 'author': 'user', 'moddate': '2024-05-22T17:27:21+09:00', 'pdfversion': '1.4', 'source': '현안과과제_주요국 경제 및 주요 가격지표 전망과 시사점_240115.pdf', 'total_pages': 23, 'page': 0, 'page_label': '1'}, page_content='24-02 2024.01.15\n 주요국 경제 및 주요 가격지표 전망과 시사점- 주요국 경기 둔화 대응책이 필요한 국내 경제')

In [21]:
# 문서 확인
chunks[1]


Document(metadata={'producer': 'Hancom PDF 1.3.0.534', 'creator': 'Hwp 2018 11.0.0.1623', 'creationdate': '2024-05-22T17:27:21+09:00', 'author': 'user', 'moddate': '2024-05-22T17:27:21+09:00', 'pdfversion': '1.4', 'source': '현안과과제_주요국 경제 및 주요 가격지표 전망과 시사점_240115.pdf', 'total_pages': 23, 'page': 1, 'page_label': '2'}, page_content='목 차■주요국 경제 및 주요 가격지표 전망과 시사점-주요국 경기 둔화 대응책이 필요한 국내')

In [23]:
## FAISS vector db 모듈 생성

import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_openai import OpenAIEmbeddings


#############################################
## 문제2: 임베딩 및 벡터DB 생성
## OpenAIEmbeddings, FAISS.from_documents를 사용한 벡터 db 생성

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# FAISS 벡터 저장소 생성

# dimension = 1536 # text-embedding-3-small 차원

# db = FAISS(
#     embedding_function=embeddings,
#     index=faiss.IndexFlatL2(dimension),
#     docstore=InMemoryDocstore(),
#     index_to_docstore_id={},
# )

# 문서추가된 벡터 저장소
vec_db = FAISS.from_documents(documents=chunks, embedding=embeddings)

#############################################

# 생성된 벡터 저장소 확인
print(vec_db)

# 저장된 문서의 인덱스-ID 매핑 확인s
print(f"총 저장된 문서 수: {len(vec_db.index_to_docstore_id)}")



총 저장된 문서 수: 154


In [24]:
## FAISS db에서 문서 검색
question_vecter_db_search_output = vec_db.similarity_search("미국 실물경제에 대해서 알려줘", k=2)

In [25]:
question_vecter_db_search_output[0].page_content

'Rule)과 미국 경기침체 시점 > < 미국 분기별 경제성장률 전망 >'

In [30]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

template = ChatPromptTemplate.from_messages([
    ('system', '당신은 직업훈련 관련 답변을 전문적으로 하는 봇입니다.'),
    ('system', '아래 content 정보를 이용해서 답변을 해주세요. 모르는 정보는 모른다고 솔직하게 이야기하세요.'),
    ('system', 'context : \n{context}'),
    ('human', '{question}')
])

question = "미국 실물 경제에 대해서 알려줘"

###########################################################
## 문제 3: 프롬프팅 및 LLM 응답 생성
## ChatPromptTemplate을 사용한 message 부분 프롬프팅 및 LLM 답변 생성

messages = template.format_messages(
    context = question_vecter_db_search_output,
    question = question
)

# messages

llm_output = llm.invoke(messages).content

###########################################################

In [32]:
### lcel 방식

template = ChatPromptTemplate.from_messages([
    ('system', '당신은 직업훈련 관련 답변을 전문적으로 하는 봇입니다.'),
    ('system', '아래 content 정보를 이용해서 답변을 해주세요. 모르는 정보는 모른다고 솔직하게 이야기하세요.'),
    ('system', 'context : \n{context}'),
    ('human', '{question}')
])

question = "미국 실물 경제에 대해서 알려줘"

chain = template | llm | StrOutputParser()

llm_output_lcel = chain.invoke({
    "context": question_vecter_db_search_output,
    "question": question
})

In [33]:
llm_output_lcel

'미국 실물 경제에 대한 구체적인 정보는 제공된 문서에서 직접적으로 다루어지지 않았습니다. 그러나 일반적으로 미국 실물 경제는 소비, 투자, 정부 지출, 수출 및 수입 등 다양한 요소로 구성됩니다. \n\n최근 몇 년간 미국 경제는 COVID-19 팬데믹의 영향으로 큰 변화를 겪었으며, 이후 회복세를 보이고 있습니다. 그러나 고강도 통화 긴축과 같은 요인으로 인해 상업용 부동산 가격이 하락하고 있으며, 이는 경제 전반에 영향을 미칠 수 있습니다. \n\n또한, 지정학적 위험과 글로벌 공급망의 변화도 미국 실물 경제에 중요한 영향을 미치는 요소입니다. 이러한 요소들은 경제 성장률, 고용률, 소비자 신뢰도 등 다양한 지표에 반영됩니다.\n\n더 구체적인 정보나 최근 데이터가 필요하다면, 경제 관련 보고서나 통계 자료를 참고하는 것이 좋습니다.'

In [31]:
llm_output

'미국 실물 경제에 대한 구체적인 정보는 제공된 문서에서 직접적으로 다루어지지 않았습니다. 그러나 일반적으로 미국 실물 경제는 소비, 투자, 정부 지출, 수출 및 수입 등 다양한 요소로 구성됩니다. \n\n최근 몇 년간 미국 경제는 COVID-19 팬데믹의 영향으로 큰 변화를 겪었으며, 이후 회복세를 보이고 있습니다. 그러나 고강도 통화 긴축과 같은 요인으로 인해 상업용 부동산 가격이 하락하고 있으며, 이는 경제 전반에 영향을 미칠 수 있습니다. \n\n또한, 지정학적 위험과 글로벌 공급망의 변화도 미국 실물 경제에 중요한 영향을 미치는 요소입니다. 이러한 요소들은 경제 성장률, 고용률, 소비자 신뢰도 등 다양한 지표에 반영됩니다.\n\n더 구체적인 정보나 최근 데이터가 필요하다면, 경제 관련 보고서나 통계 자료를 참고하는 것이 좋습니다.'